# Enhancing Network Insights: SDN Monitoring on FABRIC

This notebook was cloned from Fabric's knowledge base on [MfLib](https://github.com/fabric-testbed/jupyter-examples/tree/main/fabric_examples/mflib).
The configuration of prometheus was modified based on the requirements to push the metrics to all the nodes in the network. The used prometheus configuration is attached.

In [1]:
import os
import json
import traceback
import mflib
from mflib.mflib import MFLib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


In [ ]:
try:
    fablib = fablib_manager()
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
%%time


slice_name = "MyMonitoredSlice"

[site1,site2,site3] = fablib.get_random_sites(count=3, avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])


node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

network1_name='net1'
network2_name='net2'
network3_name='net3'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'

print(f"Setting up slice {slice_name}")
print(f"Using sites {site1}, {site2}, {site3}")

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1, image='docker_rocky_8')
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2, image='docker_rocky_8')
    iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]

    # Node3
    node3 = slice.add_node(name=node3_name, site=site3, image='docker_rocky_8')
    iface3  = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

    # Networks
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
    net3 = slice.add_l3network(name=network3_name, interfaces=[iface3], type='IPv4')

    print(f"Slice Topology Done.")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Add measurement node to topology using static method.
MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20')
print("Done")

In [ ]:
%%time
try:
    # Submit Slice Request
    print(f'Submitting the new slice, "{slice_name}"...')
    slice.submit(wait_interval=60)
    print(f'{slice_name} creation done.')

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    network1.show()
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    network2.show()
    
    network3 = slice.get_network(name=network3_name)
    network3_available_ips =  network3.get_available_ips()
    network3.show()
    
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    node1.ip_route_add(subnet=network3.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    stdout, stderr = node1.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name) 
    node2_addr = network2_available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    node2.ip_route_add(subnet=network3.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')    
    stdout, stderr = node2.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network3_name) 
    node3_addr = network3_available_ips.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=network3.get_subnet())
    
    node3.ip_route_add(subnet=network1.get_subnet(), gateway=network3.get_gateway())
    node3.ip_route_add(subnet=network2.get_subnet(), gateway=network3.get_gateway())
    
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    stdout, stderr = node3.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
mf = MFLib(slice_name)
instrumetize_results = mf.instrumentize( ["prometheus"] )

In [ ]:
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

In [ ]:
data = {}
data["get"] = ["admin_password"]
info_results = mf.info("grafana_manager", data)
print(info_results)

In [ ]:
data1 = {}
data1["get"] = ["ht_user", "ht_password"]
info_results1 = mf.info("prometheus", data1)
print(info_results1)

In [6]:
"""
scrape_configs:
  - job_name: 'federate'
    scrape_interval: 15s

    honor_labels: true
    metrics_path: '/federate'

    params:
      'match[]':
        - '{job="prometheus"}'
        - '{__name__=~"job:.*"}'

    static_configs:
      - targets:
        - 'source-prometheus-1:9090' # Target should be monitoring node
"""
# Install prometheus on all the nodes
# Update the prometheus configuration so that all the nodes receive the metrics
# This configuration should be updated on nodes(not monitoring node)

'\nscrape_configs:\n  - job_name: \'federate\'\n    scrape_interval: 15s\n\n    honor_labels: true\n    metrics_path: \'/federate\'\n\n    params:\n      \'match[]\':\n        - \'{job="prometheus"}\'\n        - \'{__name__=~"job:.*"}\'\n\n    static_configs:\n      - targets:\n        - \'source-prometheus-1:9090\' # Target should be monitoring node\n'